## インストール

### dockerのインストール
ansibleで構築。いずれ追記予定

### swarm manager設定と起動

```
docker swarm init --advertise-addr 10.136.3.183
================================================
Swarm initialized: current node (r0cr3pb7xllmom19lhx02mu0e) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-5ya5hny9r7c3lfq57a4bjaamb78maeo17jicq8s70ik94ptzfs-eiz0nl7i1ilvxy35ttwypvwii 10.136.3.183:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.
================================================
```

### swarmクラスタにノードを追加

```
### 各ノードで以下実行
$ docker swarm join --token SWMTKN-1-5ya5hny9r7c3lfq57a4bjaamb78maeo17jicq8s70ik94ptzfs-eiz0nl7i1ilvxy35ttwypvwii 10.136.3.183:2377
================================================
This node joined a swarm as a worker.
================================================

```

### 追加されたノードの確認

swarm managerのノードで実行

```
### ノードの一覧表示
$ docker node ls
================================================
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
aq3jx7y5s8h7rylc2c086woih     swarm-node2    Ready               Active
fjuw9s9xgzyt3d69byydcmrb3     swarm-node1    Ready               Active
r0cr3pb7xllmom19lhx02mu0e *   swarm-master   Ready               Active              Leader
================================================

### swarmの状態の確認
$ docker info
================================================
Containers: 1
 Running: 0
 Paused: 0
 Stopped: 1
Images: 1
Server Version: 17.06.0-ce
Storage Driver: overlay
 Backing Filesystem: xfs
 Supports d_type: false
Logging Driver: json-file
Cgroup Driver: cgroupfs
Plugins:
 Volume: local
 Network: bridge host macvlan null overlay
 Log: awslogs fluentd gcplogs gelf journald json-file logentries splunk syslog
Swarm: active
 NodeID: r0cr3pb7xllmom19lhx02mu0e
 Is Manager: true
 ClusterID: vbiiozezbop3u65rpd6ycffi1
 Managers: 1
 Nodes: 3
 Orchestration:
  Task History Retention Limit: 5
 Raft:
  Snapshot Interval: 10000
  Number of Old Snapshots to Retain: 0
  Heartbeat Tick: 1
  Election Tick: 3
 Dispatcher:
  Heartbeat Period: 5 seconds
 CA Configuration:
  Expiry Duration: 3 months
  Force Rotate: 0
 Root Rotation In Progress: false
 Node Address: 10.136.3.183
 Manager Addresses:
  10.136.3.183:2377
Runtimes: runc
Default Runtime: runc
Init Binary: docker-init
containerd version: cfb82a876ecc11b5ca0977d1733adbe58599088a
runc version: 2d41c047c83e09a6d61d464906feb2a2f3c52aa4
init version: 949e6fa
Security Options:
 seccomp
  Profile: default
Kernel Version: 3.10.0-514.26.2.el7.x86_64
Operating System: CentOS Linux 7 (Core)
OSType: linux
Architecture: x86_64
CPUs: 4
Total Memory: 3.859GiB
Name: swarm-master
ID: 2S7J:TN46:GS5K:O4PF:PPDE:N3HT:FDWE:DCMX:AG7F:4NV6:H6AI:4RSM
Docker Root Dir: /var/lib/docker
Debug Mode (client): false
Debug Mode (server): false
Registry: https://index.docker.io/v1/
Experimental: false
Insecure Registries:
 127.0.0.0/8
Live Restore Enabled: false

WARNING: overlay: the backing xfs filesystem is formatted without d_type support, which leads to incorrect behavior.
         Reformat the filesystem with ftype=1 to enable d_type support.
         Running without d_type support will not be supported in future releases.
================================================
```

## サービス作成

swarm managerのホストで実行

* オプション
    * docker service create
        * サービスを作成する
    * --replicas 1
        * 少なくとも1つのインスタンスが動いていることを希望する
    * --name
        * サービス名
    * alpine ping docker.com
        * Alpine Linuxでdocker.comにpingを実行するようにサービスが定義されている

```
### サービスを作成する
$ docker service create --replicas 1 --name helloworld alpine ping docker.com
================================================
jknp98np9ujtt9hn5rs7rizch
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.
================================================

### サービスの確認
# docker service ls
================================================
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
jknp98np9ujt        helloworld          replicated          1/1                 alpine:latest
================================================

### サービスの詳細を確認
### 「--pretty」がなければjsonフォーマットになる
# docker service inspect --pretty helloworld
================================================

ID:		jknp98np9ujtt9hn5rs7rizch
Name:		helloworld
Service Mode:	Replicated
 Replicas:	1
Placement:
UpdateConfig:
 Parallelism:	1
 On failure:	pause
 Monitoring Period: 5s
 Max failure ratio: 0
 Update order:      stop-first
RollbackConfig:
 Parallelism:	1
 On failure:	pause
 Monitoring Period: 5s
 Max failure ratio: 0
 Rollback order:    stop-first
ContainerSpec:
 Image:		alpine:latest@sha256:1072e499f3f655a032e88542330cf75b02e7bdf673278f701d7ba61629ee3ebe
 Args:		ping docker.com
Resources:
Endpoint Mode:	vip
================================================

### どのノードでサービスが動いているのか確認する
$ docker service ps helloworld
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
2a2lmia49vct        helloworld.1        alpine:latest       swarm-master   Running             Running 14 minutes ago
================================================

```


### サービスのスケール変更

```
### hello worldを5つのインスタンスで実行する状態にする
$ docker service scale helloworld=5
================================================
helloworld scaled to 5
================================================

### 起動しているホストとサービスの確認
$ docker service ps helloworld
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE                ERROR               PORTS
2a2lmia49vct        helloworld.1        alpine:latest       swarm-master   Running             Running 21 minutes ago
hbmzxlmpnhde        helloworld.2        alpine:latest       swarm-node2    Running             Running about a minute ago
tqtyi6evlybo        helloworld.3        alpine:latest       swarm-node1    Running             Running about a minute ago
uc1am1kkbqr8        helloworld.4        alpine:latest       swarm-node1    Running             Running about a minute ago
x7p58j9krnqu        helloworld.5        alpine:latest       swarm-master   Running             Running about a minute ago
================================================
```


## サービスの削除

```
### サービスの削除
$ docker service rm helloworld
================================================
helloworld
================================================

### サービスが無いことの確認
$ docker service ps helloworld
================================================
no such services: helloworld
================================================

```

## サービスのローリングアップデート

Redisを3.0.6から3.0.7にアップデートする

* オプション
    * --update-delay 10s
        * updateするときに10秒ずらして実施する
    * --update-parallelism 3
        * 最大で同時に3つのサービスをupdateできる
        * デフォルトは1サービスずつ
    * --update-failure-action
        * updateに失敗した時のアクションを指定できる
        

```
### Redis3.0.6のサービスを起動する
$ docker service create --replicas 3 --name redis --update-delay 10s redis:3.0.6
================================================
bxaeq7cs1pcrufi90z368qauq
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.
================================================

### 状況の確認
$ docker service ps redis
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE             ERROR               PORTS
2x9ns0cb26jj        redis.1             redis:3.0.6         kubernetes-node2    Running             Preparing 8 seconds ago
gfa0m5wov5lv        redis.2             redis:3.0.6         kubernetes-master   Running             Preparing 8 seconds ago
sqzq2jl9skms        redis.3             redis:3.0.6         kubernetes-node1    Running             Preparing 8 seconds ago
================================================

### ローリングアップデートを開始する
$ docker service update --image redis:3.0.7 redis
================================================
redis
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.
================================================

### 状況の確認
$ docker service ps redis
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE             ERROR               PORTS
8wvx4psiowpi        redis.1             redis:3.0.6         kubernetes-master   Running             Running 25 minutes ago
kquj43szvt2j        redis.2             redis:3.0.7         kubernetes-node1    Running             Preparing 7 seconds ago
xdvc8k4zloxh         \_ redis.2         redis:3.0.6         kubernetes-node1    Shutdown            Shutdown 6 seconds ago
e8t4dlqh0dqh        redis.3             redis:3.0.6         kubernetes-node2    Running             Running 25 minutes ago
================================================

$ docker service ps redis
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE                 ERROR               PORTS
9s1v8j1a97es        redis.1             redis:3.0.7         kubernetes-master   Running             Running about a minute ago
8wvx4psiowpi         \_ redis.1         redis:3.0.6         kubernetes-master   Shutdown            Shutdown about a minute ago
kquj43szvt2j        redis.2             redis:3.0.7         kubernetes-node1    Running             Running 2 minutes ago
xdvc8k4zloxh         \_ redis.2         redis:3.0.6         kubernetes-node1    Shutdown            Shutdown 4 minutes ago
nrwnain18s80        redis.3             redis:3.0.7         kubernetes-node2    Running             Running 26 seconds ago
e8t4dlqh0dqh         \_ redis.3         redis:3.0.6         kubernetes-node2    Shutdown            Shutdown 57 seconds ago
================================================

### サービスの詳細確認
$ docker service inspect --pretty redis
================================================
ID:		bxaeq7cs1pcrufi90z368qauq
Name:		redis
Service Mode:	Replicated
 Replicas:	3
UpdateStatus:
 State:		completed
 Started:	7 minutes ago
 Completed:	3 minutes ago
 Message:	update completed
Placement:
UpdateConfig:
 Parallelism:	1
 Delay:		10s
 On failure:	pause
 Monitoring Period: 5s
 Max failure ratio: 0
 Update order:      stop-first
RollbackConfig:
 Parallelism:	1
 On failure:	pause
 Monitoring Period: 5s
 Max failure ratio: 0
 Rollback order:    stop-first
ContainerSpec:
 Image:		redis:3.0.7@sha256:730b765df9fe96af414da64a2b67f3a5f70b8fd13a31e5096fee4807ed802e20
Resources:
Endpoint Mode:	vip
================================================

$ docker ps -a
================================================
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                      PORTS               NAMES
6864b1f9c1a6        redis:3.0.7         "docker-entrypoint..."   13 minutes ago      Up 13 minutes               6379/tcp            redis.1.9s1v8j1a97esd6cimkyouf6gv
abbc83ee4cd7        redis:3.0.6         "/entrypoint.sh re..."   42 minutes ago      Exited (0) 14 minutes ago                       redis.1.8wvx4psiowpigc69vp14rkg9q
================================================
```

## 特定のノードのコンテナを他に移す

* メンテナンスなど
* ここではswarm-node1のコンテナを他に移す

```
### 現状の確認
$ docker service ps redis
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE           ERROR               PORTS
dyhxen4d5yk3        redis.1             redis:3.0.6         swarm-node2    Running             Running 7 seconds ago
sncpngpito8z        redis.2             redis:3.0.6         swarm-master   Running             Running 7 seconds ago
zdj8hlf8q958        redis.3             redis:3.0.6         swarm-node1    Running             Running 7 seconds ago
================================================

### swarm-node1のコンテナを他に移す
$ docker node update --availability drain swarm-node1
================================================
swarm-node1
================================================

### 状況の確認
$ docker node ls
================================================
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
v9yx1vm5apm75j54j54dc36as     swarm-node2    Ready               Active
wj75qtvh5m644bl7bn6jfmcru *   swarm-master   Ready               Active              Leader
wxi6n7fhgvctraalihfinq8pf     swarm-node1    Ready               Drain
================================================

$ docker service ps redis
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
dyhxen4d5yk3        redis.1             redis:3.0.6         swarm-node2    Running             Running 44 minutes ago
sncpngpito8z        redis.2             redis:3.0.6         swarm-master   Running             Running 44 minutes ago
88os58fhkh1o        redis.3             redis:3.0.6         swarm-node2    Running             Running 6 seconds ago
zdj8hlf8q958         \_ redis.3         redis:3.0.6         swarm-node1    Shutdown            Shutdown 6 seconds ago
================================================

$ docker node inspect --pretty swarm-node1
================================================
ID:                     wxi6n7fhgvctraalihfinq8pf
Hostname:               swarm-node1
Joined at:              2017-07-24 00:22:05.341823651 +0000 utc
Status:
 State:                 Ready
 Availability:          Drain
 Address:               10.136.3.184
Platform:
 Operating System:      linux
 Architecture:          x86_64
Resources:
 CPUs:                  4
 Memory:                7.797GiB
Plugins:
 Log:           awslogs, fluentd, gcplogs, gelf, journald, json-file, logentries, splunk, syslog
 Network:               bridge, host, macvlan, null, overlay
 Volume:                local
Engine Version:         17.06.0-ce
TLS Info:
 TrustRoot:
-----BEGIN CERTIFICATE-----
MIIBaTCCARCgAwIBAgIUE0KlQJ+QSAiGTPQCZ7dwNBYDaJYwCgYIKoZIzj0EAwIw
...
ms4Cy/7mWMZmN6HQ4alAqiMvdUxR8sVTZQhjz30=
-----END CERTIFICATE-----

 Issuer Subject:        MBMxETAPBgNVBAMTCHN3YXJtLWNh
 Issuer Public Key:     MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE+LttLn/Nt...d6duAsdD541WLEcWjjqNPBtRgB7EMNZ28/jD9WA==
================================================

### ノードをActiveに戻す
$ docker node update --availability active swarm-node1
================================================
swarm-node1
================================================

### 状況の確認
$ docker node ls
================================================
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
v9yx1vm5apm75j54j54dc36as     swarm-node2    Ready               Active
wj75qtvh5m644bl7bn6jfmcru *   swarm-master   Ready               Active              Leader
wxi6n7fhgvctraalihfinq8pf     swarm-node1    Ready               Active
================================================

### コンテナはそのまま(元のホストに戻るわけでは無い)
$ docker service ps redis
================================================
ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
dyhxen4d5yk3        redis.1             redis:3.0.6         swarm-node2    Running             Running 44 minutes ago
sncpngpito8z        redis.2             redis:3.0.6         swarm-master   Running             Running 44 minutes ago
88os58fhkh1o        redis.3             redis:3.0.6         swarm-node2    Running             Running 6 seconds ago
zdj8hlf8q958         \_ redis.3         redis:3.0.6         swarm-node1    Shutdown            Shutdown 6 seconds ago
================================================
```



## ロードバランサの設定

nginxのコンテナを2付きどうしてそれぞれにアクセスするようにする

```
### サービスのportを公開する
$ docker service create --name my-web --publish 8080:80 --replicas 2 nginx
$ docker service ls
================================================
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
zv7i78yyd6tz        my-web              replicated          2/2                 nginx:latest        *:8080->80/tcp
================================================

### 公開しているportを追加する
$ docker service update --publish-add 8081:80 my-web
$ docker service ls
================================================
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
zv7i78yyd6tz        my-web              replicated          2/2                 nginx:latest        *:8080->80/tcp,*:8081->80/tcp
================================================

### 詳細情報の確認
$ docker service inspect --format="{{json .Endpoint.Spec.Ports}}"  my-web | jq
================================================
[
  {
    "Protocol": "tcp",
    "TargetPort": 80,
    "PublishedPort": 8080,
    "PublishMode": "ingress"
  },
  {
    "Protocol": "tcp",
    "TargetPort": 80,
    "PublishedPort": 8081,
    "PublishMode": "ingress"
  }
]
================================================

### 公開しているportを削除する
### portはコンテナ側を指定する
$ docker service update --publish-rm 80 my-web
$ docker service ls
================================================
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
zv7i78yyd6tz        my-web              replicated          2/2                 nginx:latest        
================================================
※ 「8081:80」で指定しても「8080:80」も一緒に消える

### udpの指定 (デフォルトはtcpのみ。udpは明示的に指定する必要がある)
$ docker service update --publish-add 8081:80/udp my-web
$ docker service ls
================================================
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
zv7i78yyd6tz        my-web              replicated          2/2                 nginx:latest        *:8081->80/udp
================================================
$ docker service update --publish-rm 80/udp my-web
$ docker service ls
================================================
ID                  NAME                MODE                REPLICAS            IMAGE               PORTS
zv7i78yyd6tz        my-web              replicated          2/2                 nginx:latest        
================================================
```

## swarmの管理

swarm managerのホストで実行

```
### tokenの表示
$ docker swarm join-token worker
================================================
To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-5ya5hny9r7c3lfq57a4bjaamb78maeo17jicq8s70ik94ptzfs-eiz0nl7i1ilvxy35ttwypvwii 10.136.3.183:2377
================================================
```

# (以下試験的な手順)

### Goのインストール

* バージョンは1.7以上
    * CentOS7はyumで入れると1.6が入ってしまうので使えない

```
$ wget https://storage.googleapis.com/golang/go1.8.linux-amd64.tar.gz
$ tar zxvf go1.8.linux-amd64.tar.gz
$ sudo chown root:root -R go
$ sudo mv go /usr/local/
$ sudo vi /etc/profile.d/go.sh
================================================
export GOROOT=/usr/local/go
export PATH=$PATH:$GOROOT/bin
================================================
$ source /etc/profile.d/go.sh

$ mkdir ~/gocode
$ vi .bashrc
================================================
export GOPATH=~/gocode
export PATH=$PATH:$GOPATH/bin
================================================
$ source ~/.bashrc

$ which go
================================================
/usr/local/go/bin/go
================================================

$ go version
================================================
go version go1.8 linux/amd64
================================================
```

### 必要なライブラリのインストール

```
### vndrのインストール
$ go get github.com/LK4d4/vndr

### golintのインストール
$ go get github.com/golang/lint/golint
```

### Swarmリポジトリをcloneする

```
### swarmをclone
$ mkdir -p $GOPATH/src/github.com/docker
$ cd $GOPATH/src/github.com/docker
$ git clone https://github.com/docker/swarm.git
$ cd swarm

### originを自分のリポジトリに指定
$ git remote remove origin
$ git remote add origin https://github.com/ftakao2007/swarm.git
$ git remote add upstream https://github.com/docker/swarm.git
$ git remote set-url --push upstream no-pushing
$ git remote -v
================================================
origin  https://github.com/ftakao2007/swarm.git (fetch)
origin  https://github.com/ftakao2007/swarm.git (push)
upstream        https://github.com/docker/swarm.git (fetch)
upstream        no-pushing (push)
================================================

### 最新を自分のforkに取り込む場合
### $ git fetch upstream master
### $ git rebase upstream/master
### $ git push -f origin master
```

### Swarmバイナリをビルドする

```
### ビルドする
$ cd $GOPATH/src/github.com/docker/swarm
$ go install .

### テスト
$ $GOPATH/bin/swarm -v
================================================
swarm version 1.2.8 (HEAD)
================================================
```

### バイナリを各ホストに配布する

``` 
$ scp $GOPATH/bin/swarm (対象ホスト):
$ ssh (対象ホスト)
$ sudo chown root:root swarm
$ sudo mv swarm /usr/local/bin
$ swarm -version
================================================
swarm version 1.2.8 (HEAD)
================================================
```

### 参考

* [公式手順](https://github.com/docker/swarm/blob/master/CONTRIBUTING.md)
* [Docker SwarmによるDockerクラスタ環境の構築（Dockerの最新機能を使ってみよう：第4回）](http://knowledge.sakura.ad.jp/knowledge/5197/)

### Etcdの設定

* swarm-managerのホストのみ実施

```
### Etcdのインストール
$ sudo yum install etcd

### 外部からのアクセスを受け付けるよう/etc/etcd/etcd.confを修正
### localhostを自分のアドレスに変更
$ cp /etc/etcd/etcd.conf /tmp
$ sudo vi /etc/etcd/etcd.conf
$ diff -u /tmp/etcd.conf /etc/etcd/etcd.conf
================================================
--- /tmp/etcd.conf      2017-07-22 11:35:46.092458672 +0900
+++ /etc/etcd/etcd.conf 2017-07-22 11:36:27.330911397 +0900
@@ -6,7 +6,7 @@
 #ETCD_HEARTBEAT_INTERVAL="100"
 #ETCD_ELECTION_TIMEOUT="1000"
 #ETCD_LISTEN_PEER_URLS="http://localhost:2380"
-ETCD_LISTEN_CLIENT_URLS="http://localhost:2379"
+ETCD_LISTEN_CLIENT_URLS="http://10.136.3.183:2379"
 #ETCD_MAX_SNAPSHOTS="5"
 #ETCD_MAX_WALS="5"
 #ETCD_CORS=""
@@ -17,7 +17,7 @@
 #ETCD_INITIAL_CLUSTER="default=http://localhost:2380"
 #ETCD_INITIAL_CLUSTER_STATE="new"
 #ETCD_INITIAL_CLUSTER_TOKEN="etcd-cluster"
-ETCD_ADVERTISE_CLIENT_URLS="http://localhost:2379"
+ETCD_ADVERTISE_CLIENT_URLS="http://10.136.3.183:2379"
 #ETCD_DISCOVERY=""
 #ETCD_DISCOVERY_SRV=""
 #ETCD_DISCOVERY_FALLBACK="proxy"
================================================
```

* 以下は全ホストで実施

```
### dockerがどの環境設定ファイルやサービス用ファイルを使っているか確認する
### 以下の場合「/usr/lib/systemd/system/docker.service」というサービス用ファイルを使っている
$ systemctl show docker | grep EnvironmentFile
================================================
(何もなし)
================================================

$ systemctl status docker | grep Loaded
================================================
   Loaded: loaded (/usr/lib/systemd/system/docker.service; enabled; vendor preset: disabled)
================================================

$ grep EnvironmentFile /usr/lib/systemd/system/docker.service
================================================
(何もなし)
================================================

### DOCKER_NETWORK_OPTIONSを追加する
### Serviceディレクティブに設定する
$ cp /usr/lib/systemd/system/docker.service /tmp
$ sudo vi /usr/lib/systemd/system/docker.service
$ diff -u /tmp/docker.service /usr/lib/systemd/system/docker.service
================================================
--- /tmp/docker.service 2017-07-22 12:04:17.007548515 +0900
+++ /usr/lib/systemd/system/docker.service      2017-07-22 12:06:11.246932774 +0900
@@ -5,6 +5,7 @@
 Wants=network-online.target

 [Service]
+EnvironmentFile=-/etc/sysconfig/docker-network
 Type=notify
 # the default is not to use systemd for cgroups because the delegate issues still
 # exists and systemd currently does not support the cgroup feature set required
================================================

### 「--cluster-store」はEtcdが動いているホストのIP、「--cluster-advertise」は各ホストのIP
$ sudo vi /etc/sysconfig/docker-network
================================================
DOCKER_NETWORK_OPTIONS='--cluster-store=etcd://10.136.3.183:2379 --cluster-advertise=10.136.3.184:2376'
================================================

### デーモンをreload
$ sudo systemctl daemon-reload

### dockerを再起動
$ sudo systemctl restart docker.service

### 設定の確認
$ systemctl show docker | grep EnvironmentFile
================================================
EnvironmentFile=/etc/sysconfig/docker-network (ignore_errors=yes)
================================================

### Etcdをインストールしているホストに接続できるかcurlで確認
$ curl -L http://10.136.3.183:2379/version
================================================
{"etcdserver":"3.1.9","etcdcluster":"3.1.0"}
================================================
```

## Swarm Managerの起動と接続確認

### Swarm Managerを起動する

```
### tcp://＜ホストのIPアドレス＞:3375 etcd://＜Etcdを稼働させているホストのIPアドレス＞:2379
### Swarm ManagerのホストにEtcdをインストールすることがほとんどなので通常は同じアドレスになるはず
$ swarm manage -H tcp://10.136.3.183:3375 etcd://10.136.3.183:2379
================================================
INFO[0000] Initializing discovery without TLS
INFO[0000] Listening for HTTP                            addr=10.136.3.183:3375 proto=tcp
================================================
※ 検証のため手で実行しているが、運用するときはデーモン化しておく


### 動作確認
docker -H tcp://10.136.3.183:3375 info
================================================
Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: swarm/1.2.8
Role: primary
Strategy: spread
Filters: health, port, containerslots, dependency, affinity, constraint, whitelist
Nodes: 0　　　※ まだnodeが接続されていないので0
Plugins:
 Volume:
 Network:
Swarm:
 NodeID:
 Is Manager: false
 Node Address:
Kernel Version: 3.10.0-514.26.2.el7.x86_64
Operating System: linux
Architecture: amd64
CPUs: 0
Total Memory: 0B
Name: kubernetes-master
Docker Root Dir:
Debug Mode (client): false
Debug Mode (server): false
Experimental: false
Live Restore Enabled: false

WARNING: No kernel memory limit support
================================================

```



### 各ホスト設定

```
### OPTIONSを追加する
### Serviceディレクティブに設定する
$ cp /usr/lib/systemd/system/docker.service /tmp
$ sudo vi /usr/lib/systemd/system/docker.service
$ diff -u /tmp/docker.service /usr/lib/systemd/system/docker.service
================================================
--- /tmp/docker.service	2017-07-23 07:52:25.115684930 +0900
+++ /usr/lib/systemd/system/docker.service	2017-07-23 07:52:48.511380621 +0900
@@ -6,6 +6,7 @@

 [Service]
 EnvironmentFile=-/etc/sysconfig/docker-network
+EnvironmentFile=-/etc/sysconfig/docker
 Type=notify
 # the default is not to use systemd for cgroups because the delegate issues still
 # exists and systemd currently does not support the cgroup feature set required
================================================

### /etc/sysconfig/dockerファイルを作成する
### アドレスは自ホストのIPアドレス
$ sudo vi /etc/sysconfig/docker
================================================
OPTIONS='-H 10.136.3.184:2375 --selinux-enabled --log-driver=journald'
================================================

### デーモンをreload
$ sudo systemctl daemon-reload

### dockerを再起動
$ sudo systemctl restart docker.service

### 設定の確認
$ systemctl show docker | grep EnvironmentFile
================================================
EnvironmentFile=/etc/sysconfig/docker-network (ignore_errors=yes)
EnvironmentFile=/etc/sysconfig/docker (ignore_errors=yes)
================================================

### 「--advertise」は自ホストのIP、「etcd」はetcdが起動しているホストのID
swarm join --advertise=10.136.3.184:2375 etcd://10.136.3.183:2379
================================================
INFO[0000] Initializing discovery without TLS
INFO[0000] Registering on the discovery service every 1m0s...  addr=10.136.3.184:2375 discovery=etcd://10.136.3.183:2379
INFO[0060] Registering on the discovery service every 1m0s...  addr=10.136.3.184:2375 discovery=etcd://10.136.3.183:2379
================================================
※ 検証のため手で実行しているが、運用するときはデーモン化しておく

### 接続確認
### IPアドレスはSwarm Managerのホストを指定
$ docker -H tcp://10.136.3.183:3375 info
================================================
Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: swarm/1.2.8
Role: primary
Strategy: spread
Filters: health, port, containerslots, dependency, affinity, constraint, whitelist
Nodes: 1
 (unknown): 10.136.3.184:2375
  m ID:
  m Status: Pending
  m Containers: 0
  m Reserved CPUs: 0 / 0
  m Reserved Memory: 0 B / 0 B
  m Labels:
  m Error: Cannot connect to the Docker daemon at tcp://10.136.3.184:2375. Is the docker daemon running?
  m UpdatedAt: 2017-07-22T03:34:09Z
  m ServerVersion:
Plugins:
 Volume:
 Network:
Swarm:
 NodeID:
 Is Manager: false
 Node Address:
Kernel Version: 3.10.0-514.26.2.el7.x86_64
Operating System: linux
Architecture: amd64
CPUs: 0
Total Memory: 0B
Name: kubernetes-master
Docker Root Dir:
Debug Mode (client): false
Debug Mode (server): false
Experimental: false
Live Restore Enabled: false

WARNING: No kernel memory limit support
================================================


```